# Point Cloud Registration

## Formulating the Problem

We know the locations of the phantom grid intersections.  We denote this set of points as $A$ where $\forall a \in A, a \in \mathbb{R}^3$.  An arbitrary grid intersection near the center of the phantom is set to be the origin.  Note that the origin is referred to as the "reference point" in the MS-12-2010 standard document.  The standard allows the reference point to be at the edge of a grid intersection instead of the center, however to keep things simple, we assume it is at the center without loss of generality.

For now, we assume that the points in $A$ form a Cartesian grid with even spacing along each dimension, $\Delta$, and that the axes of the coordinate system line up with the grid.  Thus we have

$$
\forall a \in A \;\exists\; i, j, k \in \mathbb{Z} \; \textrm{such that} \; a = \begin{bmatrix} i \Delta \\ j \Delta \\ k \Delta \end{bmatrix}.
$$

The first stage of the geometric distortion algorithm takes a 3D MRI data set and returns a set of points, which we denote as $\hat{B}$, where $\forall b \in B, b \in \mathbb{R}^3$.  The isocenter of the scanner defines the origin. 

In addition to having different origins, the frames of reference of $A$ and $\hat{B}$ may be rotated with respect to one another.  The next stage of the geometric distortion algorithm is to determine this shift and rotation that relates the frame of reference between $A$ and $\hat{B}$.  In particular, we are looking to determine the five free variables, $x$, $y$, $z$, $\theta$, and $\phi$ which define the affine transformation matrix

$$
\mathrm{S} = \mathrm{R}_x(\theta) \cdot \mathrm{R}_y(\phi) \cdot \mathrm{T}(x, y, z)
$$

where

$$
\begin{align*}
\mathrm{R}_x(\theta) &=
\begin{bmatrix}
1 & 0 & 0 & 0\\
0 & \cos(\theta) & -\sin(\theta) & 0\\
0 & \sin(\theta) & \cos(\theta) & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix},\\
\mathrm{R}_y(\phi) &=
\begin{bmatrix}
\cos(\phi) & 0 & \sin(\phi) & 0\\
0 & 1 & 0 & 0\\
-\sin(\phi) & 0 & \cos(\phi) & 0\\
0 & 0 & 0 & 1\\
\end{bmatrix}, \textrm{and}\\
\mathrm{T}(x, y, z) &=
\begin{bmatrix}
1 & 0 & 0 & x\\
0 & 1 & 0 & y\\
0 & 0 & 1 & z\\
0 & 0 & 0 & 1\\
\end{bmatrix}.\\
\end{align*}
$$


We define the set of measured grid points that have been transformed into $A$'s frame of reference to be

$$
\mathrm{B} = \left\{ \mathbf{b} \;\; \Big| \;\; \begin{bmatrix} \mathbf{b} \\ 1 \end{bmatrix} = \mathrm{S} \cdot \begin{bmatrix} \hat{\mathbf{b}} \\ 1 \end{bmatrix} , \;\; \hat{\mathbf{b}} \in \hat{B} \right\}
$$

It is useful to categorize the points in $A$ and $B$.

- We call the points in $B$ that correspond to points in $A$ matching points.
- We call the points in $B$ that do not correspond to points in $A$ false positives.
- We call the points in $A$ that do not have a corresponding point in $B$ false negatives.

Our task of finding $\mathbf{S}$ may seem hopeless, however we have some additional priori knowledge about the problem that makes it solvable.  We number these assumptions for easy reference:

**Assumption 1**: We can neglect any manufacturing error in the points of $A$.

**Assumption 2**: We can rely on the first stage of our algorithm being "reasonably good".  In other words, we can assume that some fraction of the points in $\hat{B}$ (probably at least 60%) correspond to points in $A$.

**Assumption 3**: We can assume that there is no geometric distortion for the point corresponding to origin of $A$'s frame of reference.  We can also assume that the geometric distortion is relatively small for the points closest to the origin.

**Assumption 4**: Even at the extremities of the grid the distortion is always less than $\Delta/4$ (we should do some more background research into this).

**Assumption 5**: There is no global translational error in the points $\hat{B}$--in fact, even if there was we would not be able to distinguish it from $T(x, y, z)$.  Furthermore, it does not seem to have any clinical consequence.  Thus, the mean distortion across all the points will be zero.

Note that the geometric distortion--the displacement between matching points--is what we are attempting to measure.  There are other sources of error that can shift the points in $B$.  In particular, the first stage of our algorithm will likely introduce additional errors.  Thus, we can not simply pin what we believe to be the central grid point in $B$--which we denote as $\mathbf{b}_o$ to the origin of $A$; if we did this, any error in our determination of $\mathbf{b}_o$ would introduce a systematic error into our grid distortion calculations for all the other points.  Hence, it is more robust to rely on Assumption 5, and to take into account as main of the matching points as possible when calculations $\mathrm{S}$.

**Assumption 6**: The amount of rotation required will be relatively small.  E.g. $|\theta| < 5^\circ$ and $|\phi| < 15^\circ$.

This last assumption is important, because $A$ will typically be nearly symmetric with respect to one or more axis.  Thus, it will be easy for there to be multiple possible solutions if we allow $\theta$ and $\phi$ to take on any value.  For example, imagine $A$ is a 16x16x16cube of grid points, with a couple points missing along one edge of the cube.  Further, assume that $\hat{B}$ has no false positives, and that it has a full set of "matching points"--i.e. the first stage of our algorithm was really good and found all 16x16x16 - 2 points!  In this case, the algorithm that will find $\textrm{S}$ will be solving a very fragile problem.  Rotating the set of points along any axis of the grid will likely result in a close match (only the two points will not be overlapping).

In order to avoid solving such an ill-posed problem, we simply rely on the technicians doing a reasonably good job aligning the phantom properly.  Rotation along the $x$ or $z$ directions will be constrained by the MRI table, and thus will be quite small.  Rotation along the $y$ dimension will be more likely.

In summary, given two sets of points, $A$ and $\hat{B}$, which we can make assumptions 1-6 about, we need to find $S$